# Digital Service Usage Rwanda Dataset – Data Cleaning and Preprocessing

In this notebook, we perform data preparation, cleaning, and preprocessing for the Digital Service Usage Rwanda Dataset, which contains information about citizens’ interactions with various digital services across Rwanda’s districts. The dataset captures key metrics such as the number of users reported, satisfaction scores, departments, and service names over time.

Effective data cleaning ensures that the dataset is accurate, consistent, and ready for analysis. This process helps uncover patterns in digital adoption, identify service performance gaps, and provide valuable insights for improving digital service delivery and citizen satisfaction.

Throughout this notebook, we will address common data quality issues such as missing values, duplicate records, inconsistent formatting, and invalid entries, ensuring that all columns are properly structured and formatted for analysis. We will also handle numerical and categorical variables appropriately and prepare the dataset for visualization or modeling tasks.

By the end of this preprocessing stage, the dataset will be well-structured, reliable, and analysis-ready, forming a solid foundation for any further analytics, reporting, or policy evaluation related to Rwanda’s digital transformation initiatives.


We start by importing essential Python libraries for data handling and manipulation.

- `pandas` for structured data operations.

- `numpy` for numerical operations.

- `os` for interacting with the operating system and directory structures.


In [2]:
# Import libraries

import pandas as pd
import numpy as np
import os

## 1. Define and Create Directory Paths

To ensure reproducibility and organized storage, we programmatically create directories for:

- **raw data**
- **processed data**
- **documentation**

These directories will store intermediate and final outputs for reproducibility.


In [3]:
# Get working directory
current_dir = os.getcwd()
# Go one directory up to the root directory
project_root_dir = os.path.dirname(current_dir)
# define paths to the data files
data_dir = os.path.join(project_root_dir, 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')
# Define paths to the docs folder
docs_dir = os.path.join(project_root_dir, 'docs')

# create directories if they do not exist 
os.makedirs(raw_dir, exist_ok = True )
os.makedirs(processed_dir, exist_ok = True )
os.makedirs(docs_dir, exist_ok = True)


## 2. Read in the data

We load the **Digital Service Usage Rwanda dataset** as a CSV file.

Key considerations here are:

- We ensure the file path is correctly defined using `os.path.join`.
- After loading, we inspect the first few rows to understand the structure of the dataset.


In [5]:
# Load the dataset
DSU_data_filename = os.path.join(raw_dir, "DigitalServiceUsage_Rwanda - DigitalServiceUsage_Rwanda.csv")
DSU_df = pd.read_csv(DSU_data_filename)
# Check first few rows
DSU_df.head()


,District,Service_Name,Department,Users_Reported,Satisfaction_Score_(%),Year,Month
0,Musanze,eTax Portal,Ministry of Health,910.0,NaN,2022,Unknown
1,Gasabo,Land Registration Portal,Immigration Department,3822.0,93.6,2023,Dec
2,Rusizi,eTax Portal,Rwanda Revenue Authority,516.0,82.9,2024,Mar
3,Nyagatare,E-Visa Service,Ministry of ICT,4476.0,NaN,2024,Jun
4,Nyagatare,Digital Health Records,Rwanda Revenue Authority,NaN,NaN,2023,May


We also inspect the dataset's shape. We see that the dataset has *1025* rows and *7* columns.


In [7]:
DSU_df.shape

(1025, 7)

In addition, we check the data types using `.info`. 

In [9]:
DSU_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   District                1025 non-null   object 
 1   Service_Name            1025 non-null   object 
 2   Department              1025 non-null   object 
 3   Users_Reported          501 non-null    float64
 4   Satisfaction_Score_(%)  510 non-null    float64
 5   Year                    1025 non-null   int64  
 6   Month                   937 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 56.2+ KB


## Data cleaning

## 3. Understanding the dataset

Before proceeding with the cleaning, it’s important to understand the variables in the **Digital Service Usage Rwanda Dataset**.  
This helps guide the data cleaning process and ensures that the handling of missing values, categorical variables, and data types is accurate.

**Table 1: Summary table of the variables in the dataset**

| Variable                   | Type        | Description                                                                | Values / Range (excluding NaN)           |
|:---------------------------|:------------|:---------------------------------------------------------------------------|:----------------------------------------|
| District                   | Categorical | Administrative district where the service usage was reported               | Musanze, Gasabo, Rusizi, Nyagatare, Rubavu, Burera, Nyarugenge, Nyamagabe, Kicukiro, Huye |
| Service_Name               | Categorical | Name of the digital service used by citizens                               | eTax Portal, E-Visa Service, Digital Health Records, Online Business Registration, Irembo Services, Land Registration Portal |
| Department                 | Categorical | Government department or ministry responsible for the service              | Ministry of ICT, Ministry of Health, Rwanda Revenue Authority, Immigration Department, Ministry of Lands |
| Users_Reported              | Numeric     | Number of users reported for the service                                   | 516 – 4893                              |
| Satisfaction_Score_(%)      | Numeric     | Citizen satisfaction score with the digital service (in percentage)        | 54.1 – 98.7                             |
| Year                        | Numeric     | Year when the data was recorded                                            | 2021 – 2024                             |
| Month                       | Categorical | Month when the data was recorded                                           | Apr, Feb, Jun, Aug, Sep, Dec, May, Unknown |


**Table 2: Categorical Variables Table**

| Variable         | Unique Values / Examples                             | Description                                                                 |
|:-----------------|:-----------------------------------------------------|:----------------------------------------------------------------------------|
| District         | Musanze, Gasabo, Rusizi, Nyagatare, Rubavu, Burera, Nyarugenge, Nyamagabe, Kicukiro, Huye | Represents the district where data was collected                            |
| Service_Name     | eTax Portal, Digital Health Records, E-Visa Service, Irembo Services, Online Business Registration, Land Registration Portal | Digital government service accessed by citizens                             |
| Department       | Ministry of ICT, Ministry of Health, Rwanda Revenue Authority, Ministry of Lands, Immigration Department | Ministry or institution responsible for managing the digital service        |
| Month            | Apr, Feb, Jun, Aug, Sep, Dec, May, Unknown           | Indicates the month of data collection (some months may be missing or unknown) |


## 4. Deal with missing values

Using `.isnull().sum()`, we identify columns with missing values in the dataset.  

For the **Digital Service Usage Rwanda Dataset**, the missing values are as follows:

| Column                  | Missing Values |
|:------------------------|:---------------|
| Users_Reported          | 524            |
| Satisfaction_Score_(%)  | 515            |
| Month                   | 88             |

In [11]:
DSU_df.isnull().sum()

District                    0
Service_Name                0
Department                  0
Users_Reported            524
Satisfaction_Score_(%)    515
Year                        0
Month                      88
dtype: int64

We address these:
 - For numerical columns like **Users_Reported** and **Satisfaction_Score_(%)**, missing or blank values are replaced with **0**. This is useful when missing data represents a numeric quantity and we want to avoid errors in calculations, summaries, or visualizations.
 - For categorical columns like **Month**, missing values are replaced with **Unknown** to maintain consistency and avoid issues during analysis or grouping.

In [14]:
DSU_df['Users_Reported'] = DSU_df['Users_Reported'].fillna(0)
DSU_df['Satisfaction_Score_(%)'] = DSU_df['Satisfaction_Score_(%)'].fillna(0)
DSU_df['Month'] = DSU_df['Month'].fillna('Unknown')

In [15]:
DSU_df.isnull().sum()

District                  0
Service_Name              0
Department                0
Users_Reported            0
Satisfaction_Score_(%)    0
Year                      0
Month                     0
dtype: int64

## 5. Removing Duplicates

Duplicates can distort statistical summaries and model performance. Using `.duplicated().sum()`, we count duplicate records

In [17]:
DSU_df.duplicated().sum()

25

We then inspect the duplicated records.


In [18]:
DSU_df[DSU_df.duplicated(keep = False)]

,District,Service_Name,Department,Users_Reported,Satisfaction_Score_(%),Year,Month
76,Kicukiro,E-Visa Service,Ministry of ICT,613.0,0.0,2021,Mar
136,Bugesera,eTax Portal,Ministry of Lands,0.0,0.0,2022,Mar
174,Nyagatare,E-Visa Service,Ministry of ICT,1009.0,84.8,2023,Jan
210,Kicukiro,eTax Portal,Ministry of Health,0.0,77.0,2021,Nov
280,Rusizi,Digital Health Records,Ministry of Lands,0.0,0.0,2023,Dec
319,Rubavu,Digital Health Records,Ministry of ICT,0.0,0.0,2021,Aug
371,Rwamagana,Irembo Services,Rwanda Revenue Authority,0.0,78.9,2024,May
411,Nyanza,E-Visa Service,Ministry of ICT,0.0,0.0,2023,Jul
513,Nyarugenge,Digital Health Records,Immigration Department,0.0,67.8,2021,Jul
521,Rubavu,Land Registration Portal,Ministry of Lands,0.0,68.2,2021,Mar


Finally, we remove them with `.drop_duplicates()`.

In [26]:
DSU_df.drop_duplicates(inplace=True)

We can confirm that we have no duplicates left in the dataset at this juncture.


In [27]:
DSU_df.duplicated().sum()

0

## 6. Validate Data Ranges and Numeric Formats

To ensure data consistency, I perform validation checks on numeric columns and verify that all numerical values fall within logical ranges.

Key validation steps include:

1. **Check numeric data types** — confirm that `Users_Reported` and `Satisfaction_Score_(%)` are stored as numeric values.
2. **Validate ranges** — ensure that:
   - `Users_Reported` is non-negative.
   - `Satisfaction_Score_(%)` lies between 0 and 100.
3. **Identify anomalies** — detect any unexpected or invalid values that could affect analysis.


In [29]:
DSU_df['Users_Reported'] = pd.to_numeric(DSU_df['Users_Reported'], errors='coerce')
DSU_df['Satisfaction_Score_(%)'] = pd.to_numeric(DSU_df['Satisfaction_Score_(%)'], errors='coerce')

invalid_scores = DSU_df[
    (DSU_df['Satisfaction_Score_(%)'] < 0) | (DSU_df['Satisfaction_Score_(%)'] > 100)
]

invalid_users = DSU_df[DSU_df['Users_Reported'] < 0]

print("Invalid satisfaction scores:", len(invalid_scores))
print("Invalid user counts:", len(invalid_users))
DSU_df.dtypes.head()


Invalid satisfaction scores: 0
Invalid user counts: 0


District                   object
Service_Name               object
Department                 object
Users_Reported            float64
Satisfaction_Score_(%)    float64
dtype: object

Output Explanation:
- Invalid satisfaction scores: 0
    → All satisfaction scores are in the valid range (>=0)
- Invalid user counts: 0
    → All user counts are valid (>=0)
- Column types:
    → Numeric columns are in the correct float format
    → Categorical columns are in object/string format

The dataset DSU_df is clean with no negative or invalid numeric values, and all columns have the correct data type, making it ready for analysis.

## 7. Save the Clean Dataset

Before saving the clean dataset, we re-inspect it to ensure no new issues have risen up. We first of all inspect the shape of the dataset.

In [31]:
DSU_df.shape

(1000, 7)

In [32]:
DSU_df.isnull().sum()

District                  0
Service_Name              0
Department                0
Users_Reported            0
Satisfaction_Score_(%)    0
Year                      0
Month                     0
dtype: int64

In [34]:
DSU_df.duplicated().sum()

0

The final shape of the cleaned dataset is thus *1,000* rows and *7* columns.
and there is no duplicated value

finally, we save the clean, processed dataset as a CSV file in our `processed` directory for future modelling and analysis.


In [36]:
final_file = os.path.join(processed_dir, 'DigitalServiceUsage_Rwanda_cleaned.csv') 
DSU_df.to_csv(final_file, index=False)